In [112]:
import numpy as np
import tensorflow as tf
from data_gen import *
import random
import matplotlib.pyplot as plt
import numpy as np
import math

In [113]:
num_features = 1
batch_size = 1
state_size = 2
datatype = tf.float64
num_of_operations = 2


gloabal_seed = round(random.random()*100000)
np.random.seed(gloabal_seed)
tf.set_random_seed(gloabal_seed)
sess = tf.InteractiveSession()

x,y = samples_generator(np_add, (batch_size, num_features) , (-1,1), gloabal_seed)

batchX = tf.Variable(x, dtype=datatype)
state  = tf.Variable(np.random.rand(batch_size, state_size), dtype=datatype)
W      = tf.Variable(np.random.rand(state_size+num_features, state_size), dtype=datatype, name="W")
W2     = tf.Variable(np.random.rand(state_size, num_of_operations),dtype=datatype, name="W2")
W3     = tf.Variable(np.random.rand(num_of_operations, num_features),dtype=datatype, name="W3")

batchX.initializer.run()
state.initializer.run()
W.initializer.run()
W2.initializer.run()
W3.initializer.run()

In [114]:
list(zip(x, y))

[(array([ 0.90576701]), array([ 0.90576701]))]

In [115]:

def custom_softmax(x, base = 1):
    maxx = tf.reduce_max(x, axis=1, keep_dims=True)
    print("maxx")
    print(maxx.eval())
    maxg = (x - maxx)
    print("maxg")
    print(maxg.eval())
    #powx = tf.pow(maxg, tf.cast(-base, datatype))
    maxg =tf.nn.relu(maxg)
    powx = maxg
    for i in range(base):
        powx = tf.sqrt(powx)
    #reduced  = tf.reduce_sum(powx, axis=1, keep_dims=True)

    return powx
#tf.maximum

'''
def custom_softmax( x,base = 380):
        maxx = tf.reduce_max(x, axis=1, keep_dims=True)
        powx = tf.pow(tf.cast(base, datatype), x-maxx)
        reduced  = tf.reduce_sum(powx, axis=1, keep_dims=True)
        maxg = powx/ reduced
        return maxg
'''

def calc_gradW3(W3, softmax):
    shape = W3.get_shape().as_list()
    der = tf.Variable(np.zeros(shape), dtype=datatype)
    der.initializer.run()
    for i in range(shape[0]):
        for j in range(shape[1]):
            d_temp = tf.Variable(np.zeros(shape), dtype=datatype)
            d_temp.initializer.run()
            tf.assign( d_temp[i,j], 1).eval()
            der_temp = tf.reduce_sum(- tf.matmul(softmax, d_temp))
            tf.assign( der[i,j], der_temp).eval()
    return der

In [120]:
def run_pass(batchX, state):
    X_and_state_con = tf.concat([batchX, state], 1)
    print("X_and_state_con")
    print(X_and_state_con.eval())
    layer1          = tf.matmul(X_and_state_con, W)
    print("layer1")
    print(layer1.eval())
    #next_state      = tf.nn.sigmoid(layer1)
    next_state      = tf.nn.relu(layer1)
    print("next_state")
    print(next_state.eval())
    state_dropped = tf.layers.dropout(next_state, 0, training = True)
    #print("state_dropped")
    #print(state_dropped.eval())
    logits = tf.matmul(state_dropped, W2)
    print("logits")
    print(logits.eval())
    #softmax = custom_softmax(logits, 1)
    softmax = tf.nn.softmax(logits)
    #print("cust_softmax")
    #print(cust_softmax.eval())
    print("softmax")
    print(softmax.eval())
    #softmax_loss = skewed_sigmoid_der_tf(softmax, num_of_operations)
    #print("softmax_l2_losts")
    #print(softmax_loss.eval())
    next_x = tf.matmul(softmax, W3)
    print("next_x")
    print(next_x.eval())
    selection = tf.multinomial(softmax, 1)
    #reshape = tf.reshape(selection , [batch_size, -1])
    print("selection")
    sel  = selection.eval() 
    print(sel)
    
    label = tf.one_hot(sel, num_of_operations, dtype=datatype)
    print("label")
    print(label.eval())    
    loss = tf.nn.l2_loss(label - softmax)
    print("loss")
    print(loss.eval())
    #grads  = tf.gradients(loss, [W,W2,W3])
    print("tf_discounted_epr")
    tf_discounted_epr = tf.cast( sel * 0.0, datatype)
    print(tf_discounted_epr.eval())
    optimizer = tf.train.RMSPropOptimizer(0.01)
    tf_grads = optimizer.compute_gradients(loss, var_list= [W,W2])
    tf_grads_dics = optimizer.compute_gradients(loss, var_list= [W,W2], grad_loss=tf_discounted_epr)
    train_op = optimizer.apply_gradients(tf_grads)
    print(tf_grads)
    print("weights")
    [print(w.eval()) for w in [W,W2]]
    print("grads_tf")
    [print(grad[0].eval()) for grad in tf_grads]
    print("grads_tf_disc")
    [print(grad[0].eval()) for grad in tf_grads_dics]
    print("affter applying op")
    
    [print(w.eval()) for w in [W,W2]]
    #loss fn
    #softmax = e^x/sum(e^x)
    #loss = reduced (y - softmax(relu([x;state_prev] * W1) * W2) * W3)
    #print("dev_loss_w3")
    #h = 0.001
    #dev_loss_w3 = calc_gradW3(W3, softmax)
    #print(dev_loss_w3.eval())
    return next_x, next_state

In [121]:
state1 = run_pass(batchX, state)[1].eval()

X_and_state_con
[[ 0.90576701  0.95288351  0.29508586]]
layer1
[[ 0.92405152  1.8074639 ]]
next_state
[[ 0.92405152  1.8074639 ]]
logits
[[ 1.95624412  2.49685545]]
softmax
[[ 0.36804538  0.63195462]]
next_x
[[ 0.65242329]]
selection
[[0]]
label
[[[ 1.  0.]]]
loss
0.399366638283
tf_discounted_epr
[[ 0.]]
[(<tf.Tensor 'gradients_51/MatMul_163_grad/tuple/control_dependency_1:0' shape=(3, 2) dtype=float64>, <tf.Variable 'W_1:0' shape=(3, 2) dtype=float64_ref>), (<tf.Tensor 'gradients_51/MatMul_164_grad/tuple/control_dependency_1:0' shape=(2, 2) dtype=float64>, <tf.Variable 'W2_1:0' shape=(2, 2) dtype=float64_ref>)]
weights
[[ 0.14246844  0.83353808]
 [ 0.59795942  0.96280752]
 [ 0.76324443  0.45759291]]
[[ 0.62517075  0.86004648]
 [ 0.76270078  0.94172182]]
grads_tf
[[ 0.06253999  0.04766765]
 [ 0.06579321  0.05014724]
 [ 0.02037463  0.01552943]]
[[-0.27164351  0.27164351]
 [-0.53134033  0.53134033]]
grads_tf_disc
[[ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]]
[[ 0.  0.]
 [ 0.  0.]]
affter applying op

In [ ]:
state2 = run_pass(batchX, state1)[1].eval()

In [ ]:
state3 = run_pass(batchX, state2)[1].eval()

In [ ]:
#use derivative of sigmoid to penelise for values other than 1 or 0
def sigmoid_der(x, scale = 10):
    shifted_x = x - 0.5
    scaled_x = scale*shifted_x
    res = tf.sigmoid(scaled_x)*(1 - tf.sigmoid(scaled_x))
    return (res - 0.01)*scale**3

In [ ]:
def skewed_dist(x, scale=10):
    mean = 0.5
    str_d = mean/3.5
    const = 1/tf.sqrt(2*math.pi*(str_d**2))
    power = ((x-mean)**2) / (2*str_d**2)
    return const*math.exp(-power)*scale**2

In [ ]:
def skewed_sigmoid_der(x, scale = 10, num_ops = 3):
    shifted_x = x - 0.5
    scaled_x = scale*shifted_x
    nom = tf.exp(-scaled_x)+num_ops*tf.exp(-num_ops*scaled_x)+(num_ops+1)*tf.exp(-(num_ops+1)*scaled_x)
    denom = tf.square(tf.exp(-scaled_x) + 1)*tf.square(tf.exp(-num_ops*scaled_x) + 1)
    res = nom/denom
    return (res - 0.01)*scale**3

def sigmoid(x, scale = 10, num_ops = 3):
    shifted_x = x - 0.5
    scaled_x = scale*shifted_x
    return tf.sigmoid(scaled_x)*tf.sigmoid(10*scaled_x)

In [ ]:
def plot(fn, ops):
    t = np.arange(0.0, 1.01, 0.01)
    s = [fn(val, ops).eval() for val in t]
    plt.plot(t, s)

    plt.xlabel('softmax')
    plt.ylabel('penalty')
    plt.title(fn.__name__ + " "+str(ops))
    plt.grid(True)
    plt.show()

In [ ]:
def skewed_sigmoid_der_py(x, num_ops = 3, scale = 10):
    worst_case = 1 / num_ops
    shifted_x = x - worst_case + 0.02
    scaled_x = scale*shifted_x
    nom = scale*math.exp(-scaled_x)+scale*num_ops*math.exp(-num_ops*scaled_x)
    #denom = math.square(math.exp(-scaled_x) + 1)*math.square(math.exp(-(num_ops-1)*scaled_x) + 1)
    denom = ((math.exp(-scaled_x) + 1)**2) * ((math.exp(-(num_ops-1)*scaled_x) + 1)**2)
    res = nom/denom
    return worst_case*((res)*scale**3)

def skewed_sigmoid_der_tf(x, num_ops = 3, scale = 10):
    worst_case = 1 / num_ops
    shifted_x = x - worst_case + 0.02
    scaled_x = scale*shifted_x
    nom = scale*tf.exp(-scaled_x)+scale*num_ops*tf.exp(-num_ops*scaled_x)
    denom = tf.square(tf.exp(-scaled_x) + 1)*tf.square(tf.exp(-(num_ops-1)*scaled_x) + 1)
    res = nom/denom
    return worst_case*((res - 0.01)*scale**3)

In [ ]:
plot(skewed_sigmoid_der_py, 2)
plot(skewed_sigmoid_der_py, 3)
plot(skewed_sigmoid_der_py, 4)
plot(skewed_sigmoid_der_py, 5)
plot(skewed_sigmoid_der_py, 6)

In [ ]:
plot(skewed_sigmoid_der_tf, 2)
plot(skewed_sigmoid_der_tf, 3)
plot(skewed_sigmoid_der_tf, 4)
plot(skewed_sigmoid_der_tf, 5)
plot(skewed_sigmoid_der_tf, 6)

In [ ]:
t = np.arange(0.0, 1.01, 0.01)
for val in t:
    print (val, skewed_sigmoid_der_py(val, 5, 10))

In [ ]:
import math

def sigmoid(x):
    return 1 / (1 + math.exp(-x))

def sig_inv_loss(max_error_tot):
    return (1 - sigmoid( (max_error_tot/50) - 10))

def sig_inv_loss_tf(max_error_tot):
    return (1 - tf.sigmoid( (max_error_tot/50) - 10)).eval()*0.5

def log_loss_tf(max_error_tot):
    return tf.log1p(max_error_tot*100).eval()


In [ ]:
t = np.arange(0, 100000, 100.0)
for val in t:
    print (val, log_loss_tf(val))